## Prepare example SHACL profiles from JSON API

In [1]:
import requests
import pprint
import pandas as pd

In [ ]:
profile_id = 198
profile_classes_url = 'https://ontome.net/api/classes-profile.json?lang=en&available-in-profile=' + str(profile_id)

try:
    response = requests.get(profile_classes_url)
    json_classes = response.json()
    print(len(str(json_classes)))
except Exception as e:
    print(e)


In [ ]:
json_classes

In [14]:
l = []

for cla in json_classes:
    l.append('@prefix ' + cla["namespacePrefix"] + ': ' + cla["namespaceURI"] + '.')

classes_prefixes = list(pd.unique(pd.DataFrame(l)[0]))

classes_prefixes

['@prefix crm: http://www.cidoc-crm.org/cidoc-crm/.',
 '@prefix sdh: https://sdhss.org/ontology/core/.']

In [5]:
### Classes and their elements
lc = []
for cla in json_classes:
    lc.append([cla['namespacePrefix'] , cla["namespaceURI"], cla["classIdentifierInNamespace"], 
               cla["classLabel"], cla["entityBasicType"], cla["ancestorClasses"]])

df_lc = pd.DataFrame(lc)
df_lc

,0,1,2,3,4,5
0,crm,http://www.cidoc-crm.org/cidoc-crm/,E21,Person,8,"[1, 18, 19, 64, 66, 70, 83, 375, 380, 539, 881]"
1,crm,http://www.cidoc-crm.org/cidoc-crm/,E67,Birth,9,"[1, 2, 4, 5, 83, 380, 539]"
2,crm,http://www.cidoc-crm.org/cidoc-crm/,E69,Death,9,"[1, 2, 4, 5, 83, 380, 539]"
3,sdh,https://sdhss.org/ontology/core/,C13,Geographical Place,8,"[1, 18, 64, 66, 70, 83, 375, 380, 539]"
4,sdh,https://sdhss.org/ontology/core/,C17,Construction,8,"[1, 18, 64, 65, 66, 70, 83, 375, 380, 539]"


In [6]:
profile_id = 198
profile_properties_url = 'https://ontome.net/api/properties-profile.json?lang=en&available-in-profile=' + str(profile_id)

try:
    response = requests.get(profile_properties_url)
    json_properties = response.json()
    print(len(str(json_properties)))
except Exception as e:
    print(e)


7515


In [ ]:
json_properties

In [8]:
l = []

for prop in json_properties:
    l.append('@prefix ' + prop["namespacePrefix"] + ': ' + prop["namespaceURI"] + '.')

properties_prefixes = list(pd.unique(pd.DataFrame(l)[0]))

properties_prefixes

['@prefix crm: http://www.cidoc-crm.org/cidoc-crm/.',
 '@prefix sdh: https://sdhss.org/ontology/core/.']

In [12]:
### properties and their elements
lp = []
for prop in json_properties:
    lp.append([ prop["propertyDomain"], prop["domainInstancesMinQuantifier"], prop["domainInstancesMaxQuantifier"], 
               prop['namespacePrefix'], prop['namespaceURI'], prop["propertyIdentifierInNamespace"], 
               prop["propertyRange"], prop["rangeInstancesMinQuantifier"],prop["rangeInstancesMaxQuantifier"]])

df_lp = pd.DataFrame(lp)
df_lp

,0,1,2,3,4,5,6,7,8
0,61,0,-1,crm,http://www.cidoc-crm.org/cidoc-crm/,P8,441,0,-1
1,63,0,-1,crm,http://www.cidoc-crm.org/cidoc-crm/,P8,441,0,-1
2,61,1,1,crm,http://www.cidoc-crm.org/cidoc-crm/,P98,21,0,-1
3,63,0,1,crm,http://www.cidoc-crm.org/cidoc-crm/,P100,21,1,-1
4,61,0,-1,sdh,https://sdhss.org/ontology/core/,P6,363,1,-1
5,63,0,-1,sdh,https://sdhss.org/ontology/core/,P6,363,1,-1


In [18]:
ns_list = list(set(classes_prefixes + properties_prefixes))
ns_list_txt = '\n'.join(ns_list)
print(ns_list_txt)

@prefix crm: http://www.cidoc-crm.org/cidoc-crm/.
@prefix sdh: https://sdhss.org/ontology/core/.


In [ ]:
for e in lc[:1]:
    pprint.pprint(e)

In [23]:
shacl_prof = ''

for e in lc[1:2]:
    shacl_prof += """
    sdh-shacl:{0}_PersonShape a sh:NodeShape ;
        sh:targetClass {0}:{1} ;
        rdfs:label '{2}'

        sh:property [
            sh:path rdfs:label ;
            sh:datatype xsd:string;
            sh:minCount 1;
            sh:maxCount 1;
            ];

        sh:property [
            sh:path rdfs:comment ;
            sh:datatype rdf:HTML;
            sh:minCount 1;
            sh:maxCount 1;
            ];   

        sh:property [
            sh:path [ sh:inversePath crm:P98_brought_into_life ] ;
            sh:minCount 1 ;
            sh:maxCount 1 ;
        ] ;    

    """.format(e[0], e[2], e[3])

print(shacl_prof)


    sdh-shacl:crm_PersonShape a sh:NodeShape ;
        sh:targetClass crm:E67 ;
        rdfs:label 'Birth'

        sh:property [
            sh:path rdfs:label ;
            sh:datatype xsd:string;
            sh:minCount 1;
            sh:maxCount 1;
            ];

        sh:property [
            sh:path rdfs:comment ;
            sh:datatype rdf:HTML;
            sh:minCount 1;
            sh:maxCount 1;
            ];   

        sh:property [
            sh:path [ sh:inversePath crm:P98_brought_into_life ] ;
            sh:minCount 1 ;
            sh:maxCount 1 ;
        ] ;    

    
